# Exercise 4
## Fundamentals of Machine Learning - WiSe 20/21

Authors: Elias Olofsson

Date: 2020-12-22

# Preliminaries

In [1]:
# import modules
import numpy as np
... # your code here

Ellipsis

In [2]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [3]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                rand_indices = np.random.permutation(valid_features)[:D_try]
                left, right = make_density_split_node(node, N, rand_indices)
                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)

    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        if np.all(x > self.box[0]) and np.all(x < self.box[1]): 
            return leaf.response*self.prior
        else: 
            return 0 

In [4]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. This is necessary since placing a threshold between two identical feature values
        # would create ambiguity as to which node a value is belong to.
        data_unique = np.unique(node.data[:, j])
        # Compute candidate thresholds
        tj = (data_unique[1:] + data_unique[:-1]) / 2
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # Number of instances in left and right nodes
            n_left  = (node.data[:,j] < t).sum() 
            n_right = n - n_left

            # Bounding boxes for left and right node
            m_left = m
            M_left = M.copy()
            M_left[j] = t  
            
            m_right = m.copy()
            m_right[j] = t
            M_right = M

            v_left  = np.prod(M_left-m_left) 
            v_right = np.prod(M_right-m_right)

            # Compute the errors
            loo_error_left  = n_left/(N*v_left)*(n_left/N - 2*(n_left-1)/(N-1))
            loo_error_right = n_right/(N*v_right)*(n_right/N - 2*(n_right-1)/(N-1))

            loo_error = loo_error_left + loo_error_right
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above

    m_left = m.copy()
    M_left = M.copy()
    M_left[j_min] = t_min

    m_right = m.copy()
    m_right[j_min] = t_min
    M_right = M.copy()

    # store bounding box in left and right nodes
    left.box = m_left, M_left   
    right.box = m_right, M_right

    # store data in left node -- for subsequent splits
    left.data  = node.data[(node.data[:,j_min] < t_min), :]
    right.data = node.data[(node.data[:,j_min] > t_min), :]

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right

In [5]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    m, M = node.box
    v = np.prod(M-m)
    node.response = n/(N*v)

# Decision Tree

In [6]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                rand_indices = np.random.permutation(D)[:D_try]
                left, right = make_decision_split_node(node, rand_indices)
                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return ... # your code here

In [21]:
a = np.array([1,1,1,13,3,3,3,312,1,1,2,4,4,2,3,4,3])
b, c = np.unique(a, return_counts=True)
c

array([5, 2, 5, 3, 1, 1])

In [7]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    e_min = float("inf")
    j_min, t_min = None, None

    # find best feature j (among 'feature_indices') and best threshold t for the split
    for j in feature_indices:
        data_unique = np.unique(node.data[:, j])
        
        # Compute candidate thresholds
        tj = (data_unique[1:] + data_unique[:-1]) / 2

        for t in tj:
            # Create masks for each node.
            mask_left  = (node.data[:,j] < t)
            mask_right = (node.data[:,j] > t)

            # Total number of instances in each node 
            n_left  = mask_left.sum()
            n_right = n - n_left

            # Number of instances per class in each node
            c_left, n_left_k   = np.unique(node.labels[mask_left], return_counts=True)
            c_right, n_right_k = np.unique(node.labels[mask_right], return_counts=True)

            # Calculating the Gini impurities for each node
            gini_left  = n_left  * (1 - np.sum(n_left_k**2)/n_left**2)
            gini_right = n_right * (1 - np.sum(n_right_k**2)/n_right**2)

            gini = gini_left + gini_right

            if gini < e_min:
                e_min = gini
                j_min = j
                t_min = t


    # create children
    left  = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    mask_left   = (node.data[:,j_min] < t_min)
    mask_right  = (node.data[:,j_min] > t_min)

    # data in left and right node
    left.data   = node.data[mask_left,:] 
    right.data  = node.data[mask_right,:]

    # corresponding labels in left and right node
    left.labels  = node.labels[mask_left] 
    right.labels = node.labels[mask_right] 

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left       = left
    node.right      = right
    node.feature    = j_min
    node.threshold  = t_min

    # return the children (to be placed on the stack)
    return left, right    

In [8]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = ...
    node.response = ... # your code here

In [9]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return ... # your code here

# Evaluation of Density and Decision Tree

In [10]:
# read and prepare the digits data
... # your code here

In [11]:
# train trees, plot training error confusion matrices, and comment on your results
... # your code here

# Density and Decision Forest

In [12]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [13]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [14]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here